In [1]:
from Utilities import *
import torch

import h5py
import pandas as pd
import numpy as np

In [2]:
hdf5_file_path = '/home/ms5267@drexel.edu/moberg-precicecap/data/Patient_2021-12-21_04_16.h5'
annotation_path = '/home/ms5267@drexel.edu/moberg-precicecap/data/20240207-annotations-export-workspace=precicecap-patient=7-annotation_group=90.csv'
annotation_metadata = {
    'modality':'ART'
    ,'location':'na'
    ,'scale_wrt_hd5':1e3
}
segment_length_sec = 3

data_group_name='Waveforms/ART_na'
timestamp_group_name='Waveforms/ART_na_Timestamps'
sampling_frequency=125

df_annotation = pd.read_csv(annotation_path)
df_annotation_filtered = df_annotation[(df_annotation['modality']==annotation_metadata['modality']) & (df_annotation['location']==annotation_metadata['location'])]

artifacts = df_annotation_filtered[["start_time","end_time"]].to_numpy() * int(annotation_metadata['scale_wrt_hd5'])

In [4]:
with h5py.File(hdf5_file_path, 'r') as file:
    dataset = file[data_group_name]
    timestamp = file[timestamp_group_name]

    data, timestamp = dataset[:], timestamp[:]

In [9]:
start_idx = np.searchsorted(timestamp, start_time, side='left')
end_idx = start_idx + (self.sampling_freq *  self.sample_len_sec)
# For 125Hz, the sample_len is 1250 for 10sec sample
##################################################

interval_data = self.data[start_idx:end_idx]

array([[1640364840000000, 1640364851000000],
       [1640364811000000, 1640364812000000],
       [1640364559000000, 1640364561000000],
       [1640360785000000, 1640360792000000],
       [1640360764000000, 1640360770000000],
       [1640343033000000, 1640343113000000],
       [1640336238000000, 1640336249000000],
       [1640334323000000, 1640334426000000],
       [1640325115000000, 1640325135000000],
       [1640325102000000, 1640325104000000],
       [1640321483000000, 1640321516000000],
       [1640308990000000, 1640308992000000],
       [1640283327000000, 1640283359000000],
       [1640308955000000, 1640308976000000],
       [1640283311000000, 1640283318000000],
       [1640308884000000, 1640308929000000],
       [1640283081000000, 1640283207000000],
       [1640308642000000, 1640308646000000],
       [1640266175000000, 1640266223000000],
       [1640308568000000, 1640308584000000],
       [1640265566000000, 1640265567000000],
       [1640309579000000, 1640309581000000],
       [16